In [1]:
import matplotlib.pyplot as plt
import numpy as np
import msgpack as mp
import msgpack_numpy as mpn
import os
import sys
import cv2
# directory reach

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))
from support.ar_calculations import calculate_rotmat
from support.detect_lframe import detect_lframe_from_img

# from support.pymf import get_MF_devices as get_camera_list
import toml

In [5]:
_pth = toml.load("../analysis_notebooks/DATA_PATH.toml")["data_path"]["directory"]
_pth = os.path.dirname(_pth)
_parent_folder = "validation"
_folder_name = "calibration_00"
_parent_folder = "validation"

"""pth to calibration files"""
_pth = os.path.join(_pth, "recorded_data", _parent_folder, _folder_name)
_pth

'D:\\CMC\\DeepVision\\recorded_data\\validation\\calibration_00'

In [6]:
def calculate_rotmat_from_3markers(
    corners, ids, camera_matrix, dist_coeffs, marker_length=0.05
):
    rotation_vector, translation_vector, _ = cv2.aruco.estimatePoseSingleMarkers(
        corners, marker_length, camera_matrix, dist_coeffs
    )

    ids = list(ids)

    z_inx = ids.index(80)
    org_inx = ids.index(0)
    x_inx = ids.index(49)
    print(z_inx, org_inx, x_inx)

    zvec = translation_vector[z_inx][0]
    zvec = np.reshape(zvec, (3, 1))
    org = translation_vector[org_inx][0]
    org = np.reshape(org, (3, 1))
    xvec = translation_vector[x_inx][0]
    xvec = np.reshape(xvec, (3, 1))

    translation_vector
    zvec
    rotMat = calculate_rotmat(xvec, zvec, org)
    rotMat

    t_zvec = zvec - org
    t_xvec = xvec - org

    translation_correction = np.array([0.045, -0.05, 0.045]).reshape(
        3, 1
    )  # adding the corrections in the new L frame

    rotMat.T @ t_zvec + translation_correction
    # print(t_xvec)
    print(rotMat.T @ t_xvec)
    print(rotMat.T @ t_zvec)

    return rotMat, org

In [7]:
"""read image in msgpack format"""

_webcam_frame_file = os.path.join(_pth, "webcam_frame_validation_imu_6.msgpack")
with open(_webcam_frame_file, "rb") as f:
    unpacked = mp.Unpacker(f, object_hook=mpn.decode)
    for msg in unpacked:
        webcam_frame = msg

In [8]:
with open(os.path.join(_pth, "webcam_calibration.msgpack"), "rb") as f:
    webcam_calib = mp.Unpacker(f, object_hook=mpn.decode)
    _temp = next(webcam_calib)
    _webcam_cam_mat = _temp[0]
    _webcam_dist = _temp[1]

In [9]:
"""for webcam calculating rotmat"""
corners, ids, _ = detect_lframe_from_img(webcam_frame)
w_rotmat, w_org = calculate_rotmat_from_3markers(
    corners,
    ids,
    marker_length=0.05,
    camera_matrix=_webcam_cam_mat,
    dist_coeffs=_webcam_dist,
)

2 0 1
[[ 1.52939301e-01]
 [ 0.00000000e+00]
 [-1.38777878e-17]]
[[-0.00433027]
 [ 0.        ]
 [ 0.20219804]]


In [14]:
_webcam_cam_mat

array([[727.11507074,   0.        , 510.21090502],
       [  0.        , 712.99225602, 465.45838568],
       [  0.        ,   0.        ,   1.        ]])

In [15]:
# save to path
_webcam_rotmat_file = os.path.join(_pth, "charuco_rotmat_validation_imu_6.msgpack")
with open(_webcam_rotmat_file, "wb") as f:
    _rt_packed = mp.packb(w_rotmat, default=mpn.encode)
    _org_packed = mp.packb(w_org, default=mpn.encode)
    f.write(_rt_packed)
    f.write(_org_packed)
    f.close()